In [1]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, EsmForMaskedLM
from tokenizers import Tokenizer
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle

/opt/anaconda3/envs/MachLearn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from plm_compare_esm import *
from protein_data import *

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model_name = "facebook/esm2_t33_650M_UR50D"
model, tokenizer = initialize_esm2(model_name)

Using cpu device


In [4]:
df_otc_h = pd.read_csv('/Users/johnhutchens/Desktop/Practicum/Data/DMS_ProteinGym_substitutions/OTC_HUMAN_Lo_2023.csv')

In [19]:
df_otc_h = df_otc_h.drop('DMS_score_bin', axis='columns')
df_otc_h.columns

Index(['mutant', 'mutated_sequence', 'DMS_score'], dtype='object')

In [ ]:
otc_h_esm_dict = df_otc_h.set_index(df_otc_h.columns[0]).to_dict(orient='index')

In [ ]:
otc_h_esm_dict['A102E']

{'mutated_sequence': 'MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGKSLGMIFEKRSTRTRLSTETGFELLGGHPCFLTTQDIHLGVNESLTDTARVLSSMADAVLARVYKQSDLDTLAKEASIPIINGLSDLYHPIQILADYLTLQEHYSSLKGLTLSWIGDGNNILHSIMMSAAKFGMHLQAATPKGYEPDASVTKLAEQYAKENGTKLLLTNDPLEAAHGGNVLITDTWISMGQEEEKKKRLQAFQGYQVTMKTAKVAASDWTFLHCLPRKPEEVDDEVFYSPRSLVFPEAENRKWTIMAVMVSLLTDYSPQLQKPKF',
 'DMS_score': 0.4379999999999999}

In [ ]:
for k in otc_h_esm_dict.keys():
    sequence = otc_h_esm_dict[k]['mutated_sequence']
    lp, rlp, llr = collect_log_prob_esm2(sequence, model, tokenizer)

    otc_h_esm_dict[k]['log_probs'] = lp
    otc_h_esm_dict[k]['ref_log_probs'] = rlp
    otc_h_esm_dict[k]['llr_matrix'] = llr


In [ ]:
otc_h_esm_dict['A102E']

{'mutated_sequence': 'MLFNLRILLNNAAFRNGHNFMVRNFRCGQPLQNKVQLKGRDLLTLKNFTGEEIKYMLWLSADLKFRIKQKGEYLPLLQGKSLGMIFEKRSTRTRLSTETGFELLGGHPCFLTTQDIHLGVNESLTDTARVLSSMADAVLARVYKQSDLDTLAKEASIPIINGLSDLYHPIQILADYLTLQEHYSSLKGLTLSWIGDGNNILHSIMMSAAKFGMHLQAATPKGYEPDASVTKLAEQYAKENGTKLLLTNDPLEAAHGGNVLITDTWISMGQEEEKKKRLQAFQGYQVTMKTAKVAASDWTFLHCLPRKPEEVDDEVFYSPRSLVFPEAENRKWTIMAVMVSLLTDYSPQLQKPKF',
 'DMS_score': 0.4379999999999999,
 'log_probs': tensor([[ -9.2144,  -9.6350, -10.5688,  ...,  -8.6612, -11.2060,  -9.6768],
         [ -1.9839,  -5.5226,  -6.4811,  ...,  -4.3271,  -7.0115,  -5.4861],
         [ -3.7313,  -4.4388,  -6.2629,  ...,  -4.2449,  -5.5287,  -4.8894],
         ...,
         [ -3.6619,  -5.3455,  -4.5428,  ...,  -3.5579,  -5.1311,  -3.9066],
         [ -4.2706,  -5.6263,  -5.1824,  ...,  -4.1875,  -5.6594,  -4.6486],
         [ -2.9816,  -4.6756,  -3.7991,  ...,  -2.6362,  -4.7995,  -3.2832]]),
 'ref_log_probs': tensor([[-2.2930e-03],
         [-3.8663e-01],
         [-7.4354e-01],
       

In [ ]:
filename = '/Users/johnhutchens/Desktop/Practicum/Data/OTC_Human/ems_otc_hum_matrices.pickle' 

with open(filename, 'wb') as f:
    pickle.dump(otc_h_esm_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
